The processing for stream data can be : db.collection.insertmany() for `fire`, and db.collection.insertone() for `climate`;


It seems the stream processing function is unable to finish the reference specified in the data model.
So we just insert in the collections and later set up the reference before we check the records are already there?

#### We can use tupel-slide window

In [18]:
import sys
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
import pymongo
from pymongo import MongoClient

def sendRecord1(record):

    word = record[0]
    count = record[1]
    print(word, count)
    
    client = MongoClient()
    db = client.fit5148_db
    collection = db.stream1
    collection.update({"_id": word}, {"$inc": {"count": count}}, upsert=True)
    client.close()


def sendRecord2(record):

    word = record[0]
    count = record[1]
    print(word, count)
    
    client = MongoClient()
    db = client.fit5148_db
    collection = db.stream2
    collection.update({"_id": word}, {"$inc": {"count": count}}, upsert=True)
    client.close()

# We add this line to avoid an error : "Cannot run multiple SparkContexts at once". If there is an existing spark context, we will reuse it instead of creating a new context.
sc = SparkContext.getOrCreate()

# If there is no existing spark context, we now create a new context
if (sc is None):
    sc = SparkContext()
ssc = StreamingContext(sc, 1)
ssc.checkpoint("checkpoint")



stream2 = ssc.socketTextStream(host,9999)
stream1 = ssc.socketTextStream(host,8080)

# Split each line into words
words1 = stream1.flatMap(lambda line: line.split("\n"))
words2 = stream2.flatMap(lambda line: line.split("\n"))
# Count each word in each batch
pairs1 = words1.map(lambda word: (word, 1))
pairs2 = words2.map(lambda word: (word, 1))
#wordCounts = pairs.reduceByKey(lambda x, y: x + y)

# Output the result                            
pairs1.foreachRDD(lambda rdd: rdd.foreach(sendRecord1))
pairs2.foreachRDD(lambda rdd: rdd.foreach(sendRecord2))
ssc.start()
try:
    ssc.awaitTermination(timeout=60)
except KeyboardInterrupt:
    ssc.stop()
    sc.stop()

ssc.stop()
sc.stop()
